In [2]:
from dotenv import load_dotenv
load_dotenv('myenv/.env')

True

In [6]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

model = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
)


prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)



chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)



In [9]:
chain2.invoke({"person": "obama", "language": "english"})

"The city associated with Barack Obama, the 44th President of the United States, is a bit complex due to his multicultural background. However, we can identify three cities that played significant roles in his life:\n\n1. **Honolulu, Hawaii, USA** - This is Obama's birthplace, where he was born on August 4, 1961.\n2. **Jakarta, Indonesia** - Obama spent four years (1967-1971) of his childhood in Jakarta after his mother married Lolo Soetoro, a native of Indonesia.\n3. **Chicago, Illinois, USA** - Chicago is often considered Obama's adopted hometown, as it was where he worked as a community organizer, met his future wife Michelle Obama, and began his political career.\n\nSo, to answer your question, the countries associated with these cities are:\n\n* Honolulu, Hawaii, USA - United States\n* Jakarta, Indonesia - Indonesia\n* Chicago, Illinois, USA - United States"

In [10]:
await chain2.ainvoke({"person": "obama", "language": "english"})

"The city associated with Barack Obama, the 44th President of the United States, is a bit complex due to his multicultural background. However, we can identify three cities that played significant roles in his life:\n\n1. **Honolulu, Hawaii, USA** - This is Obama's birthplace, where he was born on August 4, 1961.\n2. **Jakarta, Indonesia** - Obama spent four years (1967-1971) of his childhood in Jakarta after his mother married Lolo Soetoro, a native of Indonesia.\n3. **Chicago, Illinois, USA** - Chicago is often considered Obama's adopted hometown, as it was where he worked as a community organizer, met his future wife Michelle Obama, and began his political career.\n\nSo, to answer your question, the countries associated with these cities are:\n\n* Honolulu, Hawaii, USA - United States\n* Jakarta, Indonesia - Indonesia\n* Chicago, Illinois, USA - United States"

In [21]:
from langchain_core.runnables import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?. Return me the colour and nothing else "
)

model_parser = model | StrOutputParser()

color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4 | model_parser
)

In [22]:
question_generator.invoke("warm")

'Yellow'